In [49]:
import pandas as pd
import numpy as np

### Reformat

In [50]:
df_pre = pd.read_csv("PART 1 BEHAVIORAL SURVEY.csv", header=0, skiprows=[1, 2])
df_post = pd.read_csv("PART 2 BEHAVIORAL SURVEY.csv",  header=0, skiprows=[1, 2])

In [51]:
match_q1 = {
    "Q65": {"num": "Q1.1", "sol": "D"},
    "Q66": {"num": "Q1.2", "sol": "C"},
    "Q67": {"num": "Q1.3", "sol": "E"},
    "Q68": {"num": "Q1.4", "sol": "B"},
    "Q84": {"num": "Q1.5", "sol": "B"},
}
match_q2 = {
    "Q85": {"num": "Q2.1", "sol": 25},
    "Q86": {"num": "Q2.2", "sol": 10},
    "Q87": {"num": "Q2.3", "sol": 35},
    "Q88": {"num": "Q2.4", "sol": 32},
    "Q89": {"num": "Q2.5", "sol": 8},
}
match_q3 = {
    "Q3.1": {"Q1": 160},
    "Q3.2": {"Q17": 240, "Q2": 80},
    "Q3.3": {"Q25": 280, "Q18": 200, "Q10": 120, "Q3": 40},
    "Q3.4": {
        "Q29": 300, "Q26": 260, "Q22": 220, "Q19": 180,
        "Q14": 140, "Q11": 100, "Q4": 60, "Q7": 20,
    },
    "Q3.5": {
        "Q31": 310, "Q30": 290, "Q27": 270, "Q28": 250,
        "Q23": 230, "Q24": 210, "Q20": 190, "Q21": 170,
        "Q15": 150, "Q16": 130, "Q13": 110, "Q12": 90,
        "Q5": 70, "Q6": 50, "Q8": 30, "Q9": 10,
    },
}
match_q4 = {
    "Q4.1": {"Q4.1": 154},
    "Q4.2": {"Q4.17": 185, "Q4.2": 125},
    "Q4.3": {"Q4.18": 202, "Q4.25": 169, "Q4.10": 139, "Q4.3": 112},
    "Q4.4": {
        "Q4.22": 210, "Q4.19": 193, "Q4.29": 177, "Q4.26": 161,
        "Q4.14": 146, "Q4.11": 132, "Q4.7": 119, "Q4.4": 106,
    },
    "Q4.5": {
        "Q4.23": 215, "Q4.24": 206, "Q4.20": 197, "Q4.21": 189,
        "Q4.31": 181, "Q4.30": 173, "Q4.28": 165, "Q4.27": 158,
        "Q4.16": 150, "Q4.15": 143, "Q4.13": 136, "Q4.12": 129,
        "Q4.8": 122, "Q4.9": 116, "Q4.6": 109, "Q4.5": 103,
    },
}

In [52]:
dfnew_pre = pd.DataFrame()
dfnew_pre[["session.code", "participant.code", "participant.label"]] = df_pre[["password", "participant_code", "participant_label"]]
for key in match_q1:
    dfnew_pre[match_q1[key]["num"]] = df_pre[key]
    dfnew_pre[match_q1[key]["num"] + "_score"] = (df_pre[key] == match_q1[key]["sol"]).astype("int")
for key in match_q2:
    dfnew_pre[match_q2[key]["num"]] = df_pre[key]
    dfnew_pre[match_q2[key]["num"] + "_score"] = (df_pre[key] == match_q2[key]["sol"]).astype("int")
for key1 in match_q3:
    dfnew_pre[key1 + "_sure_offer"] = 0
    dfnew_pre[key1 + "_choice"] = np.nan
    for key2 in match_q3[key1]:
        dfnew_pre[key1 + "_sure_offer"] += df_pre[key2].notnull() * match_q3[key1][key2]
        dfnew_pre.loc[df_pre[key2].notnull(), key1 + "_choice"] = df_pre.loc[df_pre[key2].notnull(), key2]
for key1 in match_q4:
    dfnew_pre[key1 + "_today_offer"] = 0
    dfnew_pre[key1 + "_choice"] = np.nan
    for key2 in match_q4[key1]:
        dfnew_pre[key1 + "_today_offer"] += df_pre[key2].notnull() * match_q4[key1][key2]
        dfnew_pre.loc[df_pre[key2].notnull(), key1 + "_choice"] = df_pre.loc[df_pre[key2].notnull(), key2]

In [53]:
dfnew_post = df_post.iloc[[df_post["participant_id"][i] in dfnew_pre["participant.code"].values for i in range(len(df_post))]]
dfnew_post.rename(columns={"participant_id": "participant.code"}, inplace=True)

c:\Program Files\Python39\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [54]:
dfnew_pre.to_csv("PART 1 new.csv", header=True, index=None)
dfnew_post.to_csv("PART 2 new.csv", header=True, index=None)

### Text to num

In [55]:
dfnew = pd.merge(dfnew_pre, dfnew_post, on="participant.code")

In [56]:
dict_like = {
    "Not at all like me": 1,
    "Not like me": 2,
    "Somewhat not like me": 3,
    "Neutral": 4,
    "Somewhat like me": 5,
    "Like me": 6,
    "Exactly like me": 7,
}
dict_will = {
    "Completely unwilling to do so": 1,
    "Unwilling": 2,
    "Somewhat unwilling": 3,
    "Neither unwilling nor willing": 4,
    "Somewhat willing": 5,
    "Willing": 6,
    "Completely willing to do so": 7,
}
dict_patient = {
    "Very impatient person": 1,
    "Impatient person": 2,
    "Somewhat impatient": 3,
    "Neutral": 4,
    "Somewhat patient": 5,
    "Patient": 6,
    "Extremely patient": 7,
}
dict_agree = {
    "Strongly disagree": 1,
    "Disagree": 2,
    "Somewhat disagree": 3,
    "Neither agree nor disagree": 4,
    "Somewhat agree": 5,
    "Agree": 6,
    "Strongly agree": 7,
}

In [57]:
dfnew.replace({
    "Q3": dict_like,
    "Q4": dict_agree,
    "Q5": dict_agree,
    "Q6": dict_agree,
    "Q7": dict_agree,
    "Q8": dict_agree,
    "Q9": dict_agree,
    "Q10": dict_will,
    "Q11": dict_patient,
    "Q12": dict_will,
}, inplace=True)

In [59]:
dfnew.to_csv("Combined survey.csv", header=True, index=None)